In [4]:
import Collobrative_Filtering as CF
import pandas as pd
import numpy as np
import inspect
import os
import pickle
from scipy import sparse 
from sklearn.metrics import pairwise

In [ ]:
 
                
            '''    
            with open('offer_likes.pickle', 'rb') as f2:
                offer_likes = pickle.load(f2)
            with open('offerrating_table.pickle', 'rb') as f:
                rating_table = pickle.load(f)               
            with open('user_sparse.pickle', 'rb') as f1:
                tag = pickle.load(f1)
            '''
            '''
            with open('offerlabel_mapping.pickle', 'rb') as f2:
                offerlabel_mapping = pickle.load(f2)
            with open('tag_mapping.pickle', 'rb') as f3:
                tag_mapping = pickle.load(f3)
            with open('reverse_offertag.pickle', 'rb') as f4:
                reverse_offertag = pickle.load(f4)   
            with open('offer_maptb.pickle', 'rb') as f5:
                offer_maptb = pickle.load(f5)
            with open('offer_sparse.pickle', 'rb') as f6:
                offer_sparse = pickle.load(f6)
            '''

In [ ]:
with open('offerlabel_mapping.pickle', 'wb') as f2:
    pickle.dump(offer_mapping, f2, pickle.HIGHEST_PROTOCOL)
with open('tag_mapping.pickle', 'wb') as f3:
    pickle.dump(usertag_mapping, f3, pickle.HIGHEST_PROTOCOL)
with open('reverse_offertag.pickle', 'wb') as f4:
    pickle.dump(reverse_offertag, f4, pickle.HIGHEST_PROTOCOL)
with open('offer_maptb.pickle', 'wb') as f5:
    pickle.dump(offer_maptb, f5, pickle.HIGHEST_PROTOCOL)
with open('offer_sparse.pickle', 'wb') as f6:
    pickle.dump(offer_sparse, f6, pickle.HIGHEST_PROTOCOL)

In [5]:
# Load Data
file_path = inspect.getfile(inspect.currentframe())
file_direction = os.path.dirname(os.path.abspath(file_path))
offertagging = os.path.join(file_direction, 'OFFER_LOG_WITH_COUNT.xlsx')
usertagging = os.path.join(file_direction, 'MJ_TAG.xlsx')
offertagging = pd.read_excel(offertagging)
usertagging = pd.read_excel(usertagging)

In [10]:
offertagging

,ID,OFFER_ID,COUNT
0,1463,OFF0023,28
1,1218,OFF0022,18
2,1218,OFF0025,18
3,559,OFF0023,17
4,900,OFF0023,15
5,1210,OFF0023,11
6,225,OFF0023,10
7,1279,OFF0023,10
8,734,OFF0022,9
9,268,OFF0023,9


In [3]:
# Function That Define minimum numbers of data that use to build model
def threshold_likes(df, id_min, offer_min):

    # Filter data that too less
    done = False
    while not done:
        starting_shape = df.shape[0]
        offer_counts = df.groupby('ID').OFFER_ID.count()
        df = df[~df.ID.isin(offer_counts[offer_counts < offer_min].index.tolist())]
        id_counts = df.groupby('OFFER_ID').ID.count()
        df = df[~df.OFFER_ID.isin(id_counts[id_counts < id_min].index.tolist())]

        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True

    assert(df.groupby('ID').OFFER_ID.count().min() >= offer_min)
    assert(df.groupby('OFFER_ID').ID.count().min() >= id_min)
    n_users = df.ID.unique().shape[0]
    n_items = df.OFFER_ID.unique().shape[0]

    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of offers: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [4]:
def mapping_table(data):
    idx_to_tag = {}
    tag_to_idx = {}
    for (idx, tag) in enumerate(data):
        idx_to_tag[idx] = tag
    for (idx, tag) in enumerate(data):
        tag_to_idx[tag] = idx
    return idx_to_tag, tag_to_idx

In [5]:
usertag_log = usertagging.groupby(['ID','UTID']).size().reset_index(name='COUNT')
usertag_log = usertag_log.pivot_table(index=['ID'], columns='UTID',values ='COUNT').fillna(0)

In [13]:
offertag_log = offertagging.groupby(['ID','OFFER_ID','LABEL_ID']).size().reset_index(name='COUNT')
offertag_log = threshold_likes(offertag_log, 1, 1)
#offertag_log.COUNT = 1
offertag_log.head()

Ending likes info
Number of users: 310
Number of offers: 39
Sparsity: 28.040%


,ID,OFFER_ID,LABEL_ID,COUNT
0,9,OFF0025,97,40
1,9,OFF0025,229,40
2,10,OFF0004,43,30
3,10,OFF0004,47,30
4,10,OFF0004,54,60


In [18]:
offer_clicks = offertag_log[['ID','OFFER_ID', 'COUNT']].drop_duplicates()

In [27]:
offer_clicks

,ID,OFFER_ID,COUNT
0,9,OFF0025,40
2,10,OFF0004,30
4,10,OFF0004,60
18,38,OFF0011,17
26,38,OFF0019,17
34,38,OFF0020,17
39,48,OFF0007,32
47,57,OFF0007,21
55,57,OFF0025,189
57,71,OFF0023,25


In [26]:
offer_clicks.pivot_table(index='ID',columns='OFFER_ID',values='COUNT').fillna(0)

OFFER_ID,OFF0001,OFF0002,OFF0004,OFF0005,OFF0006,OFF0007,OFF0008,OFF0009,OFF0010,OFF0011,...,OFF0037,OFF0038,OFF0039,OFF0040,OFF0041,OFF0042,OFF0043,OFF0044,OFF0045,OFF0046
ID,,,,,,,,,,,,,,,,,,,,,
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
a = offertagging[['ID','OFFER_ID']].drop_duplicates()
a = a.groupby(['OFFER_ID']).size().reset_index(name='COUNT')

In [ ]:
offertag_log = offertag_log.pivot_table(index=['ID'],columns='OFFER_ID', values ='COUNT' ).fillna(0)
usertag_log = usertag_log.loc[offertag_log.index.tolist()]

In [ ]:
offertag_log.head()

In [ ]:
offertag_log = offertag_log.pivot_table(index=['ID'],columns='LABEL_ID', values ='COUNT' ).fillna(0)
usertag_log = usertag_log.loc[offertag_log.index.tolist()]

In [ ]:
# Build offer mapping table
offertags = offertag_log[['OFFER_ID','LABEL_ID']].drop_duplicates().reset_index(drop=True)
offertags['IND'] = 1
offertags = offertags.pivot_table(index='OFFER_ID',columns='LABEL_ID',values='IND').fillna(0)

In [ ]:
# Offer Sparse Matrix
offer_likes = sparse.coo_matrix(offertag_log, dtype = np.float64)
offer_likes = offer_likes.tocsr()

# User Tag Sparse Matrix
user_sparse = sparse.coo_matrix(usertag_log, dtype = np.float64)
user_sparse = user_sparse.tocsr()

# Similarity Calculation
user_similarity = pairwise.pairwise_distances(offer_likes, metric='cosine')
item_similarity = pairwise.pairwise_distances(offer_likes.T, metric='cosine')

# UserBase Rating
user_mean = offer_likes.mean(axis = 1)
user_diff = offer_likes - user_mean
user_rating = np.asarray(user_mean + user_similarity.dot(user_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T)

# ItemBase Rating
item_rating = offer_likes.dot(item_similarity) / np.array([np.abs(item_similarity).sum(axis=1)])

# Total Rating
rating_table = user_rating * item_rating

In [ ]:
user_similarity[0]

In [ ]:
user_rating

In [ ]:
offer_maptb, _ =  mapping_table(offertags.index.values)
offer_sparse = sparse.coo_matrix(offertags.values, dtype = np.float64)
offer_sparse = offer_sparse.tocsr()

In [ ]:
offertag_log = offertag_log.pivot_table(index=['ID'],columns='LABEL_ID', values ='COUNT' ).fillna(0)

In [ ]:
# Filter user tag data that only has offer tag
usertag_log = usertagging.groupby(['ID','UTID']).size().reset_index(name='COUNT')
usertag_log = usertag_log.pivot_table(index=['ID'], columns='UTID',values ='COUNT').fillna(0)

In [ ]:
offertag_log = pd.concat([usertag_log,offertag_log],keys=['p1', 'p2'], axis=1).fillna(0)['p2']


In [ ]:
offer_maptb, _ =  mapping_table(offertags.index.values)

In [ ]:
offertag_log.values.shape

In [ ]:
usertag_log = usertag_log.loc[offertag_log.index.values]

In [ ]:
offer_mapping, reverse_offertag = mapping_table(offertag_log.columns.values.tolist())

In [ ]:
offer_maptb, _ =  mapping_table(offertags.index.values)

In [ ]:
offer_mapping, reverse_offertag = mapping_table(offertag_log.columns.values.tolist())

In [ ]:
_, usertag_mapping = mapping_table(usertag_log.columns.values.tolist())

In [ ]:
# Offer Sparse Matrix
offer_likes = sparse.coo_matrix(offertag_log, dtype = np.float64)
offer_likes = offer_likes.tocsr()

# User Tag Sparse Matrix
user_sparse = sparse.coo_matrix(usertag_log, dtype = np.float64)
user_sparse = user_sparse.tocsr()

In [ ]:
offertag_log.head()

In [ ]:
offer_maptb

In [ ]:
offer_sparse.toarray()[2]

In [ ]:
sort_output(offer_sparse.toarray()[2])

In [ ]:
def sort_output(data):
    sort_data = sorted(range(len(data)), key=lambda x: data[x])
    return sort_data

In [ ]:
with open('offerrating_table.pickle', 'rb') as f:
    rating_table = pickle.load(f)
with open('user_sparse.pickle', 'rb') as f1:
    tag = pickle.load(f1)
with open('offerlabel_mapping.pickle', 'rb') as f2:
    offerlabel_mapping = pickle.load(f2)
with open('tag_mapping.pickle', 'rb') as f3:
    tag_mapping = pickle.load(f3)
with open('reverse_offertag.pickle', 'rb') as f4:
    reverse_offertag = pickle.load(f4)   
with open('offer_maptb.pickle', 'rb') as f5:
    offer_maptb = pickle.load(f5)
with open('offer_sparse.pickle', 'rb') as f6:
    offer_sparse = pickle.load(f6)

In [ ]:
rating_table[4]

In [ ]:
sort_output(rating_table[0])